<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/python/dxf_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DXF fájlok kezelése Pythonból

A DXF fájlok az AutoCAD program szöveges adatcsere formátuma.
Tartalma megegyezik a DWG fájlok tartalmával, de azokkal ellentétben a tartalma mindenki számára elérhetően dokumentált(https://docs.fileformat.com/cad/dxf/). Így tartalmuk
egy programmal is értelmezhetők illetve létrehozható.

A DXF fájlok tartalma fejezetekre bomlik (HEADER, TABLES, ENTITIES, stb.). Az egyes adatokat két sorban tartalmazza, az első sorban egy kód, a másodikban pedig egy szöveges vagy numerikus érték következik.
A kód határozza meg, hogyan kell értelmezni a következő adatot (pl. 10 - x koordináta, 2 - név blokknév/rétegnév a kontextustól függően).
A következő kis részletben (ENTITIES blokk) a kódok és értékek után megjegyzéseket írtunk, ezek nem részei a DXF fájlnak.

```
0                            ; új entitás kezdete
LINE                         ; entitás típusa
5                            ; entitás azonosító
8F4                          ; hexadecimális azonosító érték
100
AcDbEntity
8                            ; réteg
GEOD_KERITES_KAPU            ; réteg neve
6                            ; vonaltípus
Continuous                   ; folytonos
62                           ; színkód
56
100
AcDbLine
39
0.
10                           ; kezdőpont x koordináta
590408.86
20                           ; kezdőpont y koordináta
167862.39
30                           ; kezdőpont z koordináta
0.
11                           ; végpont x koordináta
590409.82
21                           ; végpont y koordináta
167861.88
31                           ; végpont z koordináta
0.
```

## DXF file létrehozása koordinátalistából

Egy CSV fájlban adott pontszámok és koordináták alapján hozzunk létre DXF fájlt.

Először letöltünk egy CSV fájlt a koordinátákkal és írjuk ki az első néhány sorát.

In [11]:
!wget -q https://raw.githubusercontent.com/OSGeoLabBp/tutorials/master/english/data_processing/lessons/code/coo_list.csv
!head coo_list.csv

﻿base_1,667568.278,271801.865,252.176
1,655738.628,257962.892,117.020
600,655738.779,257962.765,117.019
601,655739.312,257962.986,117.049
602,655921.110,257840.450,117.644
603,655922.395,257844.152,117.472
604,655925.717,257853.584,117.733
605,655928.146,257860.510,117.552
606,655930.936,257868.434,117.240
607,655933.516,257874.923,116.623


Olvassuk be a CSV fájl tartalmát egy *pandas* adatkeretbe! Mint az előző listából látható, nincs fejléc sor a fájlban.

In [2]:
import pandas as pd
names = ['id', 'east', 'north', 'elev']
fname = 'coo_list.csv'
try:
    df = pd.read_csv(fname, names=names)
except FileNotFoundError:
    print(f'Nincs ilyen fájl:{fname}')

In [13]:
df.head()

,id,east,north,elev
0,base_1,667568.278,271801.865,252.176
1,1,655738.628,257962.892,117.020
2,600,655738.779,257962.765,117.019
3,601,655739.312,257962.986,117.049
4,602,655921.110,257840.450,117.644


Hozzuk létre a DXF fájlt és írjuk ki a minimálisan szükséges bevezető információkat.

In [14]:
try:
    f_dxf = open('coo_list.dxf', 'w')
    print('  0\nSECTION\n  2\nENTITIES', file=f_dxf)
except:
    print('Nem sikerült a fájlba írni')

Menjünk végig egyesével az adatkeret sorain és írjunk ki egy pont entitást és egy szöveg entitást (pontszám) a DXF fájlba.

In [15]:
layer_p = "points"
layer_t = "point_id"
for index, row in df.iterrows():
    print(f"  0\nPOINT\n  8\n{layer_p}\n 10\n{row['east']:.3f}\n 20\n{row['north']:.3f}\n 30\n{row['elev']:.3f}", file=f_dxf)
    print(f"  0\nTEXT\n  8\n{layer_t}\n 10\n{row['east']+0.3:.3f}\n 20\n{row['north']+0.3:.3f}\n 30\n{row['elev']:.3f}\n 40\n1\n  1\n{row['id']}", file=f_dxf)

Végül zárjuk le a DXF fájlt.

In [16]:
print("  0\nENDSEC\n  0\nEOF", file=f_dxf)
f_dxf.close()

A fenti három kódblokkot, a koordinátalista beolvasását az output file megnyitását és a DXF tartalom kiírást másoljuk le a saját gépünkre és a **sys** modul  **argv** listájának segítségével a parancssorból vegyük az input file nevét. Készítsük fel a programot, hogy csővezetéken keresztül kaphassa a koordináta listát (pl. filt.py programtól).

A következő kód blokk nem fut a colab környezetben, parancssori paramétereket nem lehet átadni, a hibakezelésben az *exit()* függvényt nem hajtja végre! Ezt a saját gépünkön futtassuk.

In [ ]:
""" egy szöveges koordinátalistából DXF fájl készítése """
from sys import argv, stderr, stdin, stdout
from os import path
import pandas as pd

# parancssori paraméterek ellenőrzése
if len(argv) < 2:
    # stdin/stdout használata
    fp = stdin
    f_dxf = stdout
else:
    try:
        fp = open(argv[1])
    except:
        print(f"{argv[1]} fájl nem található")
        exit()
    dxf_name = path.splitext(argv[1])[0] + '.dxf'
    try:
        f_dxf = open(dxf_name, "w")
    except:
        print(f"{dxf_name} fájl nem hozható létre")
        exit()

# koordinátalista beolvasása
names = ['id', 'east', 'north', 'elev']

try:
    df = pd.read_csv(fp, names=names)
except:
    print(f'Nincs ilyen fájl:{argv[1]}', file=stderr)
    exit()

# DXF fájl előállítása
print('  0\nSECTION\n  2\nENTITIES', file=f_dxf)
layer_p = "points"
layer_t = "point_id"
for index, row in df.iterrows():
    print(f"  0\nPOINT\n  8\n{layer_p}\n 10\n{row['east']:.3f}\n 20\n{row['north']:.3f}\n 30\n{row['elev']:.3f}", file=f_dxf)
    print(f"  0\nTEXT\n  8\n{layer_t}\n 10\n{row['east']+0.3:.3f}\n 20\n{row['north']+0.3:.3f}\n 30\n{row['elev']:.3f}\n 40\n1\n  1\n{row['id']}", file=f_dxf)
print("  0\nENDSEC\n  0\nEOF", file=f_dxf)
if f_dxf != stdout:
    f_dxf.close()

## EZDXF Python programcsomag használata

A fenti példában közvetlenül írtuk a DXF fájlba a megfelelő kódokat, értékeket. Ennél könnyebben kezelhető megoldás, ha Python csomag használata, amikor nem kell ismernünk a kódokat, csak a megfelelő fügvényeket, metódusokat kell meghívnunk. Az EZDXF egy ilyen Python csomag.

Az EZDXF nem része a Python telepítőnek azt külön telepítenie kell a *pip* (package installer for Python) programmal.

In [1]:
!pip install -q ezdxf
import ezdxf
from ezdxf.gfxattribs import GfxAttribs

Írjuk át a *pnt2dxf.py* programunkat, hogy az EZDXF segítségével állítsuk elő a DXF fájlt.

In [4]:
doc = ezdxf.new()
doc.layers.new("points")
pnt_attr = GfxAttribs(layer="points")
doc.layers.new("point_id")
txt_attr = GfxAttribs(layer="point_id")
msp = doc.modelspace()
for index, row in df.iterrows():
    msp.add_point((row["east"], row["north"], row["elev"]), dxfattribs=pnt_attr)
    msp.add_text(row["id"], height=1, dxfattribs=txt_attr).set_placement((row["east"]+0.3, row["north"]+0.3))
doc.saveas('coo_list.dxf')

Fedezzük fel az előbb létrehozott DXF fájlt az EZDXF segítségével!

In [5]:
dxf_file = 'coo_list.dxf'
doc = ezdxf.readfile(dxf_file)  # open and load DXF
model_space = doc.modelspace()
entities = {}
# count entities
for entity in model_space:
    entity_type = entity.dxftype()
    entities[entity_type] = entities.get(entity_type, 0) + 1
for typ, count in entities.items():
    print(f"{typ:6s}: {count:6}")

POINT :    161
TEXT  :    161


Fordítsuk meg az első feladatot, azaz gyűjtsük ki egy DXF fájlból a pont elemek pozícióit egy CSV fájlba egy sorszámmal kiegészítve. Csak egy adott rétegen található pontokat vegyük figyelembe. Az adatok kikeresésére és kiírására kszítsünk egy külön függvényt! Az első példában létrehozott DXF fájlt használjuk.

In [6]:
def pnt_to_csv(msp, layer, fname):
    # msp - model space of DXF
    # layer - points on this layer are sent to CSV file
    # name of CSV file
    with open(fname, 'w') as csv_file:
        id = 1
        for entity in model_space:
            if entity.dxftype() == "POINT" and entity.dxf.layer == layer:
                print(f"{id},{entity.dxf.location[0]:.3f},{entity.dxf.location[1]:.3f},{entity.dxf.location[2]:.3f}", file=csv_file)
                id += 1

In [7]:
dxf_file = 'coo_list.dxf'
doc = ezdxf.readfile(dxf_file)  # open and load DXF
model_space = doc.modelspace()
pnt_to_csv(model_space, "points", "pnts.csv")

## Feladatok

*   Egészítse ki az első, pontlistából DXF fájl létrehozása példát, hogy az átalakítandó CSV fájl, az eredmény DXF fájl nevét és CSV fájlban használt elválasztó karaktert a paranccsorból adja meg
*   Módosítsa a pontlistából DXF fájl létrehozása példát, hogy a pont stílusát és méretét is állítsa be
*   Módosítsa a pontlistából DXF fájl létrehozása példát, hogy kezelje az elképzelhető hibákat, pl. nem létező CSV fájl, nem kezelhető szerkezetű DXF fájl, hiányzó pontszám vagy koordináta, stb.
*   Bővítse a pont koordináták DXF fájlba írását megvalósító programot, hogy a parancsorból kapja a paramétereket, több rétegről is lehessen pontokat gyűjteni, a leválogatáshoz használja az EZDXF *query* függvényét
*   Írja át a pontlistából DXF fájl létrehozása példát az EZDXF csomag használatával
*   Fedezze fel a DXF_util repozitorba található Python programokat a GitHub-on (https://github.com/zsiki/dxf_utils/blob/master/python) és alkalmazza azokat
*   Készítsen újabb programot a DXF_util repozitori bővítésére

